<div align="center">  <img src="http://image-bed-zodiac.oss-cn-qingdao.aliyuncs.com/18-10-30/48416734.jpg" width="1000"/> </div><br>

因为要帮导师在 GitHub 上搭建个人主页，需要用到 GitHub Pages 可我 GitHub 帐号的 github.io 仓库已经开了博客，只能再申请一个GitHub 帐号用于存放导师的主页仓库，当我想 push 文件的时候却提示 `access denied`，报错信息显示我使用了账户 A 却在往账户 B 的仓库 push 文件，遂尝试解决。

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#失败方法" data-toc-modified-id="失败方法-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>失败方法</a></span></li><li><span><a href="#基于-ssh-key-的多账户切换" data-toc-modified-id="基于-ssh-key-的多账户切换-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>基于 ssh key 的多账户切换</a></span><ul class="toc-item"><li><span><a href="#确认本机是否已有ssh-key" data-toc-modified-id="确认本机是否已有ssh-key-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>确认本机是否已有ssh key</a></span></li><li><span><a href="#生成-ssh-key" data-toc-modified-id="生成-ssh-key-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>生成 ssh key</a></span></li><li><span><a href="#添加-ssh-key" data-toc-modified-id="添加-ssh-key-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>添加 ssh key</a></span></li><li><span><a href="#生成配置文件" data-toc-modified-id="生成配置文件-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>生成配置文件</a></span></li><li><span><a href="#配置本地仓库" data-toc-modified-id="配置本地仓库-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>配置本地仓库</a></span></li></ul></li></ul></div>

# 失败方法

上网查了一下，原来因为之前设置了`git config --global credentianl.helper store` 将输入的用户密码保存了下来，相关设置保存在 git 配置文件 `~/.gitconfig` 中。既然账户不对，自然就想到更改全局账户。

按照网上的说法，操作了一遍

首先，查看 git 全局账户

```shell
git config --global -l
git config --system -l
```

释放 credential.helper

```shell
git config --global --unset credential.helper
git config --system --unset credential.helper
```

重新设置全局账户

```shell
git config --global user.name [username]
git config --global user.email [email]
```

如果有必要，设置仓库本地账户，并设置保存密码

```shell
git config --local user.name [username]
git config --local user.email [email]
git config --local credential.helper store
```

可选操作

```shell
git remote -v # 查看远端地址
git remote set-url origin https://github.com/username/reponame # 设置远端地址
```


**但是，当我将这些都操作完了之后，在我更改了全局和本地账户之后，我 push 的时候还是提示 `access denied`，git 仍然在使用我最原始的帐号，查了一下似乎是git 缓存了登录信息，而仅仅更改全局和本地帐号并不能更改登录信息。**

至于如何删除这个登录信息的缓存，暂时没有找到，遂放弃继续使用 https 加帐号密码的登录方式，转投 ssh 怀抱。

# 基于 ssh key 的多账户切换

## 确认本机是否已有ssh key

首先确认自己系统内有没有 ssh key

```
cd ~/.ssh
ls *.pub # 列出所有公钥文件
```

id_rsa.pub

若有，确认使用当前 key 或者生成新 key，若没有，生成新 key。由于我需要登录两个帐号，所以在已经存在的 key 的基础上，再生成一个 key

## 生成 ssh key

```shell
ssh-keygen -t rsa -C "841611653@qq.com"
```
-t 是使用的算法，一般都选择 rsa -C 为备注，一般以邮箱作为备注，提示保存文件时以 `id_rsa_work` 保存，用于区分第一个密钥`id_rsa`

至此，我的 `~/.ssh` 文件夹内共有4个文件

```
.
├── id_rsa
├── id_rsa_work
├── id_rsa_work.pub
├── id_rsa.pub
└── known_hosts
```

其中以 `.pub` 结尾的为公钥文件，用于填写在GitHub网站，同名不带 `.pub` 后缀的为私钥文件，自己保存不要泄漏，`known_hosts` 文件为已知 host 缓存，目的是加快 ssh 访问速度。

## 添加 ssh key

### 为 GitHub 添加 ssh key

登录 GitHub 在账户 `Settings > SSH and GPG keys > New SSH key > Add key` 将之前生成的 ssh key 的 pub 文件中的内容粘贴进去，另一个账户也如法炮制。

### 为本机添加 ssh key

```shell
ssh-add ~/.ssh/id_rsa
ssh-add ~/.ssh/id_rsa_work
```

如果成功会看到 Identity added 的反馈。



## 生成配置文件

生成配置文件用于根据情况选择不同的 ssh key

```shell
touch ~/.ssh/config
nano config
```

写入内容

```
#Default GitHub
Host github.com
  HostName github.com
  User git
  IdentityFile ~/.ssh/id_rsa
  
#Work GitHub
Host github.com-work
   HostName github.com
   User git
   IdentityFile ~/.ssh/id_rsa_work
```

两个配置不同的地方在于 Host 和 IdentityFile，第二处 Host 填写的 github.com-work 是为了区分默认配置

## 配置本地仓库

对于本地已存在的仓库，查看远端

```shell
git remote -v
```

添加/更改远端

```shell
git remote add origin git@github.com-work:username/reponame.git
# git remote set-url origin git@github.com-work:username/reponame.git
```

git 就会依据 Hostname 选择需要的账户进行 push 和 pull 了

References:
> - [Quick Tip: How to Work with GitHub and Multiple Accounts](https://code.tutsplus.com/tutorials/quick-tip-how-to-work-with-github-and-multiple-accounts--net-22574)
- [How to manage multiple GitHub accounts on a single machine with SSH keys](https://medium.freecodecamp.org/manage-multiple-github-accounts-the-ssh-way-2dadc30ccaca)